# Daten explorieren und bereinigen

In diesem Notebook lernst du
+ wie man Daten in Python einliest und wie man anschließend mit diesen arbeiten kann.
+ wie man die Daten für die Bereinigung vorbereitet.
+ wie man gezielt nach fehlerhaften Daten sucht und wie man mit diesen umgehen kann.

## Einbinden von Bibliotheken

Wir beginnen damit die **Bibliotheken**, die wir später benötigen werden, zu importieren. Eine Bibliothek (in Python) ist eine Sammlung von nützlichen Funktionen und Werkzeugen. In unserem Fall helfen sie uns bei beim Einlesen, Manipulieren und Visualisieren der Daten.

In [ ]:
# Bibliotheken zur Speicherung und Manipulation der Daten in sogenannten DataFrames (Tabellen)
import pandas as pd 
import numpy as np

# Bibliotheken zur Visualisierung der Daten
import cufflinks as cf
from ipywidgets import interactive
cf.go_offline()
cf.set_config_file(offline=False, world_readable=True)

## Kennenlernen der Daten

Wir haben bereits den pandas-Datentyp `DataFrame` kennengelernt und Daten aus einer Excel-Tabelle in ein solches DataFrame übertragen. Für das folgende Notebook stehen uns die Parkplatzdaten der Firma ASP aus dem Jahre 2017 zur Verfügung. Du findest Sie in dem Ordner `Daten` unter dem Namen `Einlaeufe2017.csv`.

![Liboriberg](Ressourcen/parkplaetze.jpg)

Die Excel-Tabelle beginnt mit einem TimeStamp, in dem Tag, Monat, Jahr und die volle Stunden der Tageszeit kodiert sind. Für die verschiedenen Parkflächen folgt dann eine Belegungszahl, die Ein- und Ausfahrten der aktuellen Stunde und zwei Korrekturwerte, die nur ab und zu mit Werten belegt sind. Das Zählen der Ein- und Ausfahrten erfolgt über Induktionsschleifen an der Einfahrt/Ausfahrt des jeweiligen Parkplatzes. Die einzelnen Parkflächen können nicht separat erfasst werden.

![Parkplatzdaten-Excel](Ressourcen/Parkplatzdaten-Excel.png)

Durch einen Export dieser Tabelle in das CSV-Format erhalten wir eine Textdatei in deren ersten Zeile die Spaltennamen der Excel-Tabelle, durch Semikolon getrennt, abgelegt sind. In den folgenden Zeilen stehen dann die Daten, die jeweils durch den TimeStamp eingeleitet werden und dann je Spalte durch Semikolon getrennt werden.

![Parkplatzdaten-CSV](Ressourcen/Parkplatzdaten-CSV.png)

Auf der Internetseite von ASP finden wir eine Übersicht über die Zuordnung der Parkplätze und deren Kapazitäten

| Parkplatz | Standort | Kapazität |
| :--- | :--- | :--- |
| P1 | Tiefgarage Königsplatz | 640 |
| P2 | Parkplatz Florianstraße | 290 |
| P3 | Parkhaus Neuhäuser Tor | 540 |
| P41 & P42 | Parkplatz Paderhalle / Maspernplatz | 670 |
| P5 | Parkpalette Rolandsweg | 540 |
| P6 | Parkhaus Liborigalerie | 500 |
| P7 | Parkplatz Le-Mans-Wall/Liboriberg | 290 |
| P8 | Tiefgarage Volksbank / Theater | 160 |

## Einlesen der Daten

Die uns vorliegende CSV-Datei kann jetzt mit dem bereits behandelten `pandas` Kommando `read_csv(...)` in ein `DataFrame` eingelesen werden.

Neben dem Dateipfad müsssen wir auch noch das Trennzeichen (`sep` für *seperator*) angeben, mit dem die einzelnen Spalten voneinander getrennt wurden.

In [ ]:
# Einlesen der Daten
df = pd.read_csv(filepath_or_buffer='Daten/Einlaeufe2017.csv', sep =';')

## Überblick verschaffen

### Anzeige `display`

Wir können uns den Inhalt des `DataFrames` mit Hilfe der Funktion `display(...)` anzeigen lassen.

In [ ]:
display(df)

Alternativ können wir auch nur den Namen verwenden (Kurzschreibweise). Intern wird dann automatisch die Funktion `display` aufgerufen.

In [ ]:
df

Diese Art der Anzeige ist in der Regel sehr lang.

Mit der Methode `head()` des `DataFrames` können wir uns auf die **ersten fünf Zeilen** unserer Daten beschränken.

In [ ]:
df.head()

### Datentypen

Mit dem Property `dtypes` des `DataFrames` bekommen wir eine Übersicht über die Datentypen der einzelnen Tabellenspalten.

In [ ]:
df.dtypes

Pandas weist jeder Spalte beim Einlesen automatisch einen Typ zu.

Pandas unterscheidet dabei zwischen folgenden Datentypen:

| Pandas dtype | Python type | Verwendung |
| :--- | :--- | :--- |
| object | str | Text |
| int64 | int | Ganze Zahlen |
| float64 | float | Fließkommazahlen |
| bool | bool | Wahrheitswerte |
| datetime64 | datetime64 (numpy) | Datum und Uhrzeit |
| timedelta[ns] | NA | Abstände zwischen zwei Zeitpunkten |
| category | NA | Endliche Liste von Kategorien |

<div style="background-color: lightblue; padding: 5px 20px 20px">

### Aufgabe (Datentypen 1)

Stellt mit Hilfe der Tabelle eine Vermutung auf, warum den Spalten unterschiedliche Datentypen zugewiesen wurden.
</div>

*Hier ist Platz für deine Antwort*

### Kennzahlen

Die Methode `describe()` liefert einen Überblick über wichtige statistische Kennzahlen wie z.B. die Standardabweichung oder den Durchschnitt der Werte.

In [ ]:
df.describe()

<div style="background-color: lightblue; padding: 5px 20px 20px">
    
### Aufgabe (Kennzahlen)

Erklärt unter dem Aspekt der Datentypen, warum in der Übersicht, die wir mittels `describe()` erhalten haben, nicht alle Spalten auftauchen.
</div>

*Hier ist Platz für deine Antwort*

Wir kommen später noch auf das Problem mit den Datentypen zurück.

## Daten vorbereiten (Standardisierung)

Bevor wir mit der eigentlichen Bereinigung den Daten (Schritt 10) beginnen, müssen wir die Daten dahingehend zunächst vorbereiten (Schritt 5-8). Das bedeutet, dass wir die Daten **formatieren** und **strukturieren**. Man spricht dabei auch von der **Standardisierung** der Daten.

Beim Explorieren der ersten fünf Zeilen und mit einem Blick auf die Datentypen der einzelnen Spalten fallen gleich zwei größere Probleme ins Auge:

1. Die Spalte `TimeStamp` enthält Angaben über **Datum und Uhrzeit** der Messung und sollte daher den Datentyp `timestamp` zugewiesen bekommen.
2. Ganz rechts befinden sich unbenannte und offenbar **leere Spalten**. Diese sollen später in unserer Auswertung nicht auftauchen.

Das erste Problem können wir lösen, indem wir manuell mit Hilfe der folgenden Anweisung die Spalte `TimeStamp` als Zeitstempel (`datetime`) interpretieren.

In [ ]:
# Versuche die Spalte TimeStamp als DateTime zu interpretieren
# dayfirst=True bewirkt, dass das Format DD-MM-YYYYY verwendet wird
df['TimeStamp'] = pd.to_datetime(df['TimeStamp'], dayfirst=True)

Das zweite Problem lösen wir, indem wir alle Spalten löschen, die komplett leer sind (nur den Wert NA = *not assigned* enthalten).

In [ ]:
# Lösche alle Spalten (axis=1), die leer sind
df = df.dropna(how='all', axis=1)

<div style="background-color: lightblue; padding: 5px 20px 20px">

### Aufgabe (Überprüfung)

Überprüft, ob die Bereinigung erfolgreich war, indem ihr die Datentypen und das `DataFrame` anzeigen lasst.
</div>

In [ ]:
# Hier ist Platz für deine Lösung

## Daten indizieren

Ein **Index** (oder auch Schlüssel) identifiziert jede Zeile eines `DataFrames` eindeutig. Derzeit werden die Zeilen noch anhand der Zeilennummer (Position) identifiziert.

Da unsere Daten einem zeitlichen Ablauf (TimeSeries) folgen liegt es nahe, die Spalte `TimeStamp` als Index zu wählen.

In [ ]:
# Wähle die Spalte TimeStamp als neuen Index
df = df.set_index('TimeStamp')

# Daten anhand des Zeitstempels chronologisch sortieren
df = df.sort_index()

Wir können nun auf die Daten zu einem bestimmten Zeitpunkt durch Angabe des entsprechenden `TimeStamp` zugreifen:
```python
df['01-01-2017 01:00']
```

Wir werden später sehen, dass uns die Wahl des Index die graphische Darstellung erleichtert, da die Spalte TimeStamp nun automatisch auf der x-Achse dargestellt wird.

Wenn wir uns das DataFrame nun erneut anschauen, sehen wir, dass die Spalte `TimeStamp` nun eingerückt ist.

In [ ]:
df.head()

Desweiteren taucht sie nun nicht mehr in der Liste der Spaltennamen auf.

In [ ]:
df.columns

<div style="background-color: lightblue; padding: 5px 20px 20px">

### Aufgabe (Slicing)

Gib mit Hilfe des Index alle Daten zwischen dem 01.04.2017 und dem 03.05.2017 aus. Nutze hierfür den [`:`-Operator](https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html).
</div>

In [ ]:
# Hier ist Platz für deine Lösung

## Daten auf die Parkplatzbelegungen beschränken

Da aus technischen Gründen die stündlichen Belegungszahlen der Parkplätze in der Regel nicht zu den Ein- und Ausfahrten des zugehörigen Parkplatzes passen, werden wir in den folgenden Auswertungen diese zusätzlichen Informationen ignorieren und alleine mit den Belegungszahlen der 8 Parkplätze weiterarbeiten.

Dazu wählen wir alle Spalten aus, die den Ausdruck `AK` enthalten.

In [ ]:
# Auswahl aller Spalten, deren Namen die Zeichenkette AK beinhalten
df_AK = df.filter(like = 'AK')
# Ausgabe zur Kontrolle
df_AK.head()

## Das Problem mit dem Datentyp

In Schritt 3 haben wir festgestellt, dass nicht allen Spalten automatisch der Datentype `float64` zugewiesen wurde. Das liegt daran, dass die entsprechenden Spalten **nicht-numerische Werte (NaN)** und Sonderzeichen  (z.B. `-`) beinhalten.

Das ist beispielsweise dann problematisch, wenn wir auf den Daten mathematisch operieren möchten. Möchten wir beispielsweise die **relative Belegung** des Parkplatzes ermitteln, müssen wir den **absoluten Wert** durch die **maximale Kapazität** des Parkplatzes dividieren.

In den Spalten, die derzeit den Datentyp `object` haben kommt es dabei zu einem so genannten **TypeError**, da die Division auf nicht-numerischen Werten nicht definiert ist.

**Beachte:** Die Parkplätze können auch überbelegt sein. Das ist dann der Fall, wenn Fahrzeuge auf einen vollen Parkplatz einfahren und nach einem Parkplatz suchen.

In [ ]:
# Berechnet die prozentuale Belegung aus der absoluten Anzahl der Fahrzeuge
def prozentuale_belegung(belegung):
    return belegung / 670

# Das Anwenden der Funktion auf die Spalte P1AK erzeugt einen Fehler
# Diesen fangen wir vorsichtshalber ab und geben ihn aus
try:
    df_AK['P1AK'].apply(func=prozentuale_belegung)
except Exception as e:
    print(type(e))
    print(e)

Zunächst möchten wir feststellen, welche **Zeilen** betroffen sind. Dazu listen wir alle Zeilen auf, in denen **nicht-numerische Werte** stehen.

In [ ]:
# Zeigt alle Zeilen an, in denen mindestens ein nicht-numerischer Wert steht

# Wir wenden die Funktion pd.to_numeric auf df_AK mit dem Parameter coerce an. Diese versucht alle Werte in numerische
# Werte umzuwandeln. Die Werte, die nicht als numerische Werte interpretiert werden können (z.B. '-') werden zu Null ('coerce').
# .notnull.all(1) liefert alle Zeilen (0 = Spalten, 1 = Zeilen), die KEINEN Nullwert haben. Der Operator '~' invertiert
# nun die Auswahl und liefert dementsprechend alle Zeilen, in denen mindestens einmal Null auftaucht, also in denen
# mindestens ein nicht numerischer Wert steht.

df_AK[~df_AK.apply(pd.to_numeric, errors='coerce').notnull().all(1)]

Nachdem wir die entsprechenden Zeilen identifiziert haben, können wir nun überlegen, wie wir mit den entsprechenden Werten verfahren möchten.

Eine sinnvolle Möglichkeit ist es den Datentyp `float64` zu erzwingen und alle nicht interpretierbaren Werte als `NaN` zu kennzeichnen.

Dieses Verfahren erlaubt es uns numerische Operationen auf den Spalten durchzuführen. Die NaN-Werte werden dabei einfach ignoriert.

Wir verwenden dafür den `pandas`-Befehl `.to_numeric(...)` mit dem Parameter `errors='coerce`, welcher festlegt, wie mit nicht interpretierbaren Werten umgegangen werden soll. 

Wir beginnen mit der Spalte `P1AK`.

In [ ]:
# Erzwingt einen numerischen Datentyp (float64) und b als NaN
df_AK['P1AK'] = pd.to_numeric(df_AK['P1AK'], errors='coerce')

Eine Überprüfung der Datentypen zeigt, dass nun auch die Spalte P1AK den Datentyp `float64` hat.

In [ ]:
df_AK.dtypes

Nun erzeugt auch die Division keinen Fehler mehr.

In [ ]:
df_AK['P1AK'].apply(func=prozentuale_belegung)

<div style="background-color: lightblue; padding: 5px 20px 20px">

### Aufgabe (Datentypen 2)

Ändere nun auch den Datentyp der übrigen Spalten in `float64`. Dabei kannst du entweder jede Spalte einzelnd ändern oder dir überlegen, wie man alle Spalten gleichzeitig ändern kann. Überprüfe deine Lösung indem du die Datentypen ausgibst (`.dtypes`).
</div>

In [ ]:
# Hier ist Platz für deine Lösung

## Erste Visualisierung

Nachdem wir die Daten nun ausreichend vorbereitet haben können wir damit beginnen, die Daten zu visualisieren.

Dazu verwenden wir den Befehl `.iplot()`. Dieser ist Teil der `cufflinks`-Bibliothek und erzeugt einen so genannten **Plot** (Graphen) mit Hilfe von `plotly`.

In [ ]:
df_AK.iplot()

## Daten bereinigen

Offensichtlich enthält unser `DataFrame` Daten, die vollkommen **unrealistisch** erscheinen. So liegt die maximale Belegung des Parkplatzes P4.1 bei circa 40.000. 

Wir müssen uns nun überlegen wie wir mit diesen fehlerhaften Daten umgehen. In der Regel gibt es in einem solchen Fall keine perfekte Lösung.

Eine Möglichkeit ist es einen sinnvollen Höchstwert zu ermitteln und alle Werte die darüber liegen "abzuschneiden". Dabei müssen wir Folgendes beachten:

1. Es gibt keine maximale Belegung, da sich prinzipiell mehr Fahrzeuge auf einem Parkplatz aufhalten können als Parkplätze vorhanden sind. Die Wahl eines Höchstwerts ist daher pragmatisch.
2. Wir wissen nicht, ob der Parkplatz zum jeweiligen Zeitpunkt tatsächlich voll war. Ein technischer Fehler könnte auch bei einem leeren Parkplatz zu solchen Werten führen.
3. Uns fehlt es an Hintergrundinformationen über die Verteilung der Kapazitäten auf die einzelenen Parkplätze (z.B. bei P4)

Unter anderem aus diesen Gründen ersetzen wir die unrealistische Werte stattdessen durch `NaN`.

Wir ermitteln zunächst den höchsten Wert, der unser Meinung nach noch als realistisch erscheint. Dazu **sortieren** wir die Werte **absteigend** und entscheiden uns dann von Fall zu Fall.

In [ ]:
# Setze die Anzahl anzuzeigender Spalten auf 100
pd.options.display.max_rows = 100
# Sortiere die Werte absteigend
df_AK['P41AK'].sort_values(ascending=False)

Die Wahl eines Schwellenwertes ist nicht eindeutig. Dennoch scheint der Wert **656** noch im realistischem Bereich und liegt gleichzeitg deutlich unter dem nächst höheren Wert von 3072.

Wir wählen alle Werte aus, die kleiner oder gleich 656 sind und ersetzen alle anderen durch **NaN**.

In [ ]:
condition = df_AK['P41AK'] <= 656
df_AK['P41AK'].where(cond=condition, other=np.nan, inplace=True)

In [ ]:
df_AK['P42AK'].sort_values(ascending=False, na_position='last')

<div style="background-color: lightblue; padding: 5px 20px 20px">
    
### Aufgabe (Schwellenwert)

Verfahre auf die selbe Weise mit den übrigen Spalten. Welche Spalten benötigen einer Überarbeitung? Entscheide dich jeweils für einen sinnvollen Schwellenwert.
</div>

In [ ]:
# Hier ist Platz für deine Lösung

### Überprüfung

Zum Schluss überprüfen wir unsere Ergebnisse, indem wir die Daten erneut Plotten.

In [ ]:
df_AK.iplot()

Um einen besseren Überblick über die Daten zu bekommen eignet sich die Darstellung in einem so genanntem **Boxplot**. Dort können wir direkt den Median, das Minimum und das Maximum sowie das **q1-Quartil** und das **q3-Quartil** ablesen und erhalten so einen Überblick über die Verteilung der Daten.

In [ ]:
df_AK.iplot(kind='box')

## Daten für die Auswertung speichern

Die vollständig bereinigten Daten speichern wir nun für die spätere Verwendung unter einem neuen Namen ab.

In [ ]:
df_AK.to_csv('Daten/EinlauefeBereinigt.csv')